In [5]:
!pip install discord

ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [3]:
import discord

In [10]:
# bot.py
import os
import time

import discord
#from dotenv import load_dotenv
from discord.ext import commands
import asyncio

import random

game_channel_id = 846682471511883776



def printCustom(mode, text):
    space = 8 - len(mode)
    print("["+mode.capitalize()+"]"+space*" "+text)


printCustom("info", "Program started")

TOKEN = "ODQ1NDA3NjQ3MjQxOTk0MzEw.YKghOg.X6IY5O7_USiDLkR-W_OWE_c10bo"
GUILD = "Serveur de TheSun"


bot = commands.Bot(command_prefix='-')

#GAME VARIABLES
channel = -1


@bot.event
async def on_ready():
    global channel
    print("xxxxxxxxx")
    channel = bot.get_channel(game_channel_id)



TWSS = {}
TWSS["twss_message"] = ""
TWSS["votes"] = {}
TWSS["members"] = []
TWSS["themes"] = ["in the beach", "at school", "in the bus"]
TWSS["game_theme"] = ""
TWSS["game"] = 0
TWSS["nb_games"] = 5
TWSS["messages"] = []
TWSS["last_turn_select_message"] = -1
TWSS["twss_game_started"] = False

@bot.command(name="twss")
async def twss(ctx, nb_games:int):
    global channel, TWSS
    TWSS["nb_games"] = nb_games
    msg = await channel.send("React if you want to start the game")
    await msg.add_reaction("✅")
    print(TWSS["twss_message"])
    TWSS["twss_message"] = msg


@bot.command(name="start")
async def twss(ctx):
    global channel, TWSS
    #GET ALL USERS THAT REACTED

    #GET THE -twss MESSAGE
    msg = await channel.fetch_message(TWSS["twss_message"].id)

    reaction = msg.reactions[0]
    async for user in reaction.users():
        print(user.name)
        if not (user in TWSS["members"]) and (user.name != "Nori"):
            TWSS["members"].append(user)
            TWSS["votes"][user.name] = {"mark":0}

    print(TWSS["members"])
    TWSS["twss_game_started"] = True
    #START game
    TWSS["game_theme"] = random.choice(TWSS["themes"])
    msg = await channel.send("The theme is: "+TWSS["game_theme"])
    msg = await channel.send("The turn if for player: "+TWSS["members"][TWSS["game"] % len(TWSS["members"])].name)
    TWSS["last_turn_select_message"] = msg
    #await TWSS()


async def TWSS_GAME(message):
    global channel, TWSS

    #if the message recieved is from the person who sent the message
    if message.author == TWSS["members"][TWSS["game"] % len(TWSS["members"])]:
        TWSS["messages"].append(message)
        await TWSS["last_turn_select_message"].delete()
        await message.add_reaction("✅")
        await message.add_reaction("❌")

        TWSS["game"] += 1

        if TWSS["game"] != TWSS["nb_games"]:
            msg = await channel.send("The turn if for player: "+TWSS["members"][TWSS["game"] % len(TWSS["members"])].name)
            TWSS["last_turn_select_message"] = msg
        else:
            print("GAME ENDED")
            print(TWSS["votes"])
            #collect all reactions
            for msg in TWSS["messages"]:
                fetched_msg = bot.fetch_message(id=msg.id)
                reactions = fetched_msg.reactions
                for react in reactions:
                    if react.emoji == "✅":
                        TWSS["votes"][msg.author.name]["mark"] += ((react.count - 1)*10)
                    elif react.emoji == "❌":
                        TWSS["votes"][msg.author.name]["mark"] -= ((react.count - 1)*10)

            

            embed=discord.Embed(title="That's what she said results",
                                description="The results of the \"TWSS\" game theme \""+TWSS["game_theme"]+"\"",
                                color=0x008080)
            embed.set_author(name="Nori")
            for user in members:
                embed.add_field(name=user.name, value="Points: "+TWSS["votes"][user.name]["mark"], inline=False)
            await channel.send(embed=embed)

            TWSS["twss_game_started"] = False


















@bot.event
async def on_message(message):
    global TWSS_CONFIG, members

    if message.author == bot.user:
        return 0


    if message.content[0] == "-":
        await bot.process_commands(message)

    else:
        print(message.content)
        if TWSS["twss_game_started"]:
            await TWSS_GAME(message)



"""if __name__ == '__main__':
    try:
        bot.run(TOKEN)
        print("jjjjjjjjj")
    except KeyboardInterrupt:
        bot.logout()
        bot.close()
    finally:
        print("Disconnected")"""

bot.run(TOKEN)

[Info]    Program started


RuntimeError: ignored